Imports

In [30]:
import pandas as pd
import numpy as np
import re
import langid

from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer

Import review data

In [31]:
reviews = pd.read_csv('/Users/satanhaha/Documents/MS ADS/ML 1/ML1 Final Project/Airbnb-Renting-Optimizer/data_raw/reviews.csv.gz')

EDA

Omit
- reviewer_id 
- reviewer_name
- id (review comment id)

In [32]:
reviews.drop(columns=['reviewer_id','reviewer_name','id'], inplace=True)

Clean
- remove weird characters
- lowercase
- remove urls
- remove extra whitespace
- keep punctuation mostly


In [33]:
# remove <br/>
reviews['comments'] = reviews['comments'].astype(str).str.replace("<br/>", "", regex=False)

# lowercase
reviews['comments'] = reviews['comments'].astype(str).str.lower()

# remove extra whitespace
reviews['comments'] = [re.sub(r'\s+', ' ', review).strip() for review in reviews['comments']]

# add whitepsace after punctuation
reviews['comments'] = [re.sub(r'([.!?,])([A-Za-z])', r'\1 \2', review) for review in reviews['comments']]

# replaces thank you patterns with just thanks
reviews['comments'] = [re.sub(r'\b(thanks|thank you)\s+[A-Z][a-z]+\b', 'thanks', review) for review in reviews['comments']]

% of Spanish reviews

In [ ]:
# sample = reviews['comments'].sample(10000, random_state=1)

# def detect_lang(text: str) -> str:
#     text = text.strip()
#     if len(text) < 20:
#         return "short"
#     lang, score = langid.classify(text)
#     return lang

# lang_counts = sample.apply(detect_lang).value_counts(dropna=False)
# lang_pct = (lang_counts / lang_counts.sum()).round(4)

# print(lang_counts.head(15))
# print(lang_pct.head(15))

comments
en       9163
short     424
es        180
fr         64
de         46
pt         28
it         16
zh         15
nl         10
ru          7
pl          7
ko          5
da          3
la          3
an          3
Name: count, dtype: int64
comments
en       0.9163
short    0.0424
es       0.0180
fr       0.0064
de       0.0046
pt       0.0028
it       0.0016
zh       0.0015
nl       0.0010
ru       0.0007
pl       0.0007
ko       0.0005
da       0.0003
la       0.0003
an       0.0003
Name: count, dtype: float64


Topic modeling using BERT 
- embedding happens implicitly

Fit BERT model on train subset

In [ ]:
vectorizer_model = CountVectorizer(
    stop_words="english",
    ngram_range=(1, 2),
    min_df=50
)

topic_model = BERTopic(
    min_topic_size=100,
    calculate_probabilities=True,
    verbose=True,
    vectorizer_model=vectorizer_model
)

# subset train reviews
train_reviews = reviews['comments'].sample(100000, random_state=1)

# fit bert on train reviews ~88 min
topics_train, probs_train = topic_model.fit_transform(train_reviews.tolist())

2026-02-20 02:25:16,507 - BERTopic - Embedding - Transforming documents to embeddings.
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1942.05it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches:   0%|          | 1/3125 [00:02<1:51:15,  2.14s/it]


KeyboardInterrupt: 

Save BERT model

In [25]:
# save model
topic_model.save('review_topic_model')

# load
# topic_model = BERTopic.load("airbnb_topic_model")

2026-02-20 02:02:26,252 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
topic_model.get_topic_info()

Assign topics to entire batch
- unsupervised feature engineering; so not worried about data leakage

In [ ]:
batch_size = 100000

for i in range(0, len(reviews), batch_size):

    batch = reviews['comments'].iloc[i:i+batch_size].tolist()

    topics_batch, probs_batch = topic_model.transform(batch)

    out = reviews.iloc[i:i+batch_size].copy()
    out['topic'] = topics_batch

    out.to_csv(f'topics_batch_{i}.csv', index=False)

    print(f'Saved batch {i}')

Aggregate to listing level

Merge to master file
- make new features, where each feature is a cluster that is discoered via clustering algorithm
- ex: property A has safety score of 1/10, clean score of 5/10, etc
